# Uso de HDFS como sistema de Archivos distribuidos

Dataset: https://www.ruoa.unam.mx/csv_data/more/minuto.php

In [ ]:
# Importamos las librerías de dask y distributed

import dask.dataframe as dd
from dask.distributed import Client
import dask, dask_ml
import distributed

Lo primero que se debe hacer es configurar nuestra conexión para que los workers con los que trabajemos sepan cómo tienen que manejar los datos

In [ ]:
dask.config.set({'distributed.scheduler.allowed-failures': 99})
dask.config.set({"array.chunk-size": "256 MiB"})
dask.config.set({"distributed.workers.memory.spill": 0.85})
dask.config.set({"distributed.workers.memory.target": 0.75})
dask.config.set({"distributed.workers.memory.terminate": 0.98})

## Conectarse al servidor

Nos conectamos al servidor donde está montado el HDFS, esta dirección suele cambiar ya que depende de dondé se haya montado el sistema


In [ ]:
client = Client('tcp://132.248.203.7:8786')

## Leer los datos

Dask además de leer grandes vólumenes de datos, también puede leer una carpeta completa montada en HDFS, esto siempre y cuando todos los archivos tengan el mismo formato, es decir, que tengan las mismas columnas con el mismo tipo de dato.

Por ejemplo en este caso se leerán todos los archivos csv en la carpeta user/michell21 los cuales representan las mediciones por hora del clima en Morelia desde 2015 hasta mediados del 2021, tomadas por la RUOA

In [ ]:
ddf = dd.read_csv('hdfs://132.248.203.7:9820/user/michell21/*.csv')

Una vez leídos los datos, ser pueden realizar las mismas operaciones que se hacen en un dataframe de pandas.

In [ ]:
ddf.mean().compute()

In [ ]:
ddf.dtypes

In [ ]:
ddf.head(10)

In [ ]:
ddf.columns

In [ ]:
ddf["duration"].count().compute()

In [ ]:
# Este comando sirve para liberar la memoria de Dask, eliminando el Dataframe cargado al inicio (No borra los datos del HDFS)
client.cancel(ddf)

## Conclusión

El uso de Dask + HDFS es muy útil cuando se tiene que trabajar con grandes cantidades de datos, pero es importante hacer un  analisis sobre el uso del mismo ya que en ciertos casos este puede ser perjudicial para el procesamiento de datos. 
Para evitar que el uso de HDFS nos juegue en contra, hay que poner atención lo siguiente
- Las características con las que cuenta nuestro equipo de computo. Si el analisis se puede ejecutar en un equipo local es preferible optar por este.
- El peso de el(los) archivo(s) a analizar. Si es un archivo que pese varios Gigas o son decenas de archivos a procesar, es buena idea el uso de HDFS. 
- La velocidad de la conexión a internet (en caso de ser un servidor remoto)
